### Ray Life


In [1]:
import ray
import numpy as np
import time

In [3]:
@ray.remote
class RoLSubGrid(object):
        
    # Create an actor each with its own data
    def __init__(self, dim, has_glider=False):
        self.local_grid = np.zeros(shape=[dim+2,dim+2],dtype=np.uint8)
        if has_glider==True:
            glider = np.array([[0,1,0],[0,0,1],[1,1,1]], dtype=np.uint8)
            self.local_grid[1:glider.shape[0]+1,1:glider.shape[1]+1] = glider
        
    def set_neighbors(self, top, bottom, left, right, tl, tr, bl, br):
        self.tl = tl
        self.bl = bl
        self.tr = tr
        self.br = br
        self.left = left
        self.right = right
        self.top = top
        self.bottom = bottom
        
    def get_neighbors(self):
        return self.top, self.bottom, self.left, self.right, self.tl, self.tr, self.bl, self.br
    
    def get_grid(self):
        return self.local_grid
    
    def set_grid(self, value):
        self.local_grid = np.ones(shape=[dim+2,dim+2],dtype=np.uint8)
        
    
    def rules_of_life(self):
        """Evaluate the rules of life on a 2-d subarray.
        The array should have an overlap of 1 cell in all dimension 
        and on the corner.
    
        Args:    

        Returns:
            outgrid (ndarray): Array updated by rules of life 
        """
        # we will receive an array with 1 dimension of padding
        xdim, ydim = self.local_grid.shape
    
        # output array to keep updates
        outgrid = np.zeros(shape=self.local_grid.shape, dtype=self.local_grid.dtype)
    
        # update only in center (non-overlapping) regaion
        for x in range(1,xdim-1):
            for y in range(1, ydim-1):
                sum = self.local_grid[x-1,y-1] + self.local_grid[x,y-1] + self.local_grid[x+1,y-1] +\
                      self.local_grid[x-1,y]   + self.local_grid[x+1,y] +\
                      self.local_grid[x-1,y+1] + self.local_grid[x,y+1] + self.local_grid[x+1,y+1]
                # three neighbors birth
                if (sum == 3):
                    outgrid[x,y] = 1
                # two neighbors no change
                elif (sum == 2):
                    outgrid[x,y] = self.local_grid[x,y]
                    # <2 or >3 death
                else:
                    outgrid[x,y] = 0 
                    
        return outgrid

    def step(self):
        """Update the glider locally. Then, update the edges and corners from previous iterations.
        
        Args:
            None
        Returns:
            None
        """
        self.local_grid = self.rules_of_life()
        
        # trim to the grid without overlap
        #trimmed = self.local_grid[1:-1,1:-1]
#        self.local_grid = np.zeros(shape=self.local_grid.shape, dtype=self.local_grid.dtype)
#        self.local_grid[1:-1,1:-1] = trimmed 
        
        # remove return
        # return self.local_grid
        
    def cut(self, xl, xh, yl, yh):
        trimmed = self.local_grid[1:-1,1:-1]
        #trimmed = np.array([[ 255, 1, 2, 3, 4, 5, 6, 7,],
        #           [10,11,12,13,14,15,16,17],
        #           [20,21,22,23,24,25,26,27],
        #           [30,31,32,33,34,35,36,37],
        #           [ 0, 1, 2, 3, 4, 5, 6, 7,],
        #           [10,11,12,13,14,15,16,17],
        #           [20,21,22,23,24,25,26,27],
        #           [30,31,32,33,34,35,36,37]])
        return trimmed[xl:xh,yl:yh] 
    
    def exchange(self):
        tloid = self.tl.cut.remote(7,8,7,8)  #tl cuts br
        troid = self.tr.cut.remote(7,8,0,1)  #tr cuts bl
        bloid = self.bl.cut.remote(0,1,7,8)  #bl cuts tr
        broid = self.br.cut.remote(0,1,0,1)  #br cuts tl
        toid = self.top.cut.remote(7,8,0,8)  #top
        boid = self.bottom.cut.remote(0,1,0,8)  #bottom
        loid = self.left.cut.remote(0,8,7,8)  #left
        roid = self.right.cut.remote(0,8,0,1)  #right
        
        self.local_grid[0,0] = ray.get(tloid)  #tl cuts br
        self.local_grid[0,9] = ray.get(troid)  #tr cuts bl
        self.local_grid[9,0] = ray.get(bloid)  #bl cuts tr
        self.local_grid[9,9] = ray.get(broid)  #br cuts tl
        self.local_grid[0,1:9] = ray.get(toid)  #top
        self.local_grid[9,1:9] = ray.get(boid)  #bottom
        self.local_grid[1:9,0:1] = ray.get(loid)  #left
        self.local_grid[1:9,9:10] = ray.get(roid)  #right
        return

In [4]:
ray.init(num_cpus=4, ignore_reinit_error=True)

# Sleep a little to improve the accuracy of the timing measurements used below,
# because some workers may still be starting up in the background.
time.sleep(2.0)

In [9]:
# Create  workers

dim = 2

oids = np.empty([dim,dim], dtype=object)
roids = []

# create grids
for ix in range(dim):
    for iy in range(dim):
        if ix == 0 and iy == 0:
            oids[ix, iy] = RoLSubGrid.remote(8, True)
        else:
            oids[ix, iy] = RoLSubGrid.remote(8, False)
        #print(f"({ix},{iy}) {oids[ix,iy]}")

# set neighbors T B L R TL TR BL BR            
for ix in range(dim):
    for iy in range(dim):
        roids.append(oids[ix,iy].set_neighbors.remote( 
                    oids[(iy-1)%dim,ix], oids[(iy+1)%dim,ix], oids[iy,(ix-1)%dim], oids[iy,(ix+1)%dim],
                    oids[(iy-1)%dim,(ix-1)%dim], oids[(iy-1)%dim,(ix+1)%dim], oids[(iy+1)%dim,(ix-1)%dim], oids[(iy+1)%dim,(ix+1)%dim]))

for ix in range(dim):
    for iy in range(dim):
        ray.wait(roids)
        print(ray.get(oids[ix,iy].get_neighbors.remote()))

(Actor(RoLSubGrid, 3f73735202639b76a894b9d201000000), Actor(RoLSubGrid, 3f73735202639b76a894b9d201000000), Actor(RoLSubGrid, 3787a2cc318bf98b2df6daaa01000000), Actor(RoLSubGrid, 3787a2cc318bf98b2df6daaa01000000), Actor(RoLSubGrid, 620c7863bd6d6f5d38c784c901000000), Actor(RoLSubGrid, 620c7863bd6d6f5d38c784c901000000), Actor(RoLSubGrid, 620c7863bd6d6f5d38c784c901000000), Actor(RoLSubGrid, 620c7863bd6d6f5d38c784c901000000))
(Actor(RoLSubGrid, 970ed42ec911b726a3f27e4e01000000), Actor(RoLSubGrid, 970ed42ec911b726a3f27e4e01000000), Actor(RoLSubGrid, 620c7863bd6d6f5d38c784c901000000), Actor(RoLSubGrid, 620c7863bd6d6f5d38c784c901000000), Actor(RoLSubGrid, 3787a2cc318bf98b2df6daaa01000000), Actor(RoLSubGrid, 3787a2cc318bf98b2df6daaa01000000), Actor(RoLSubGrid, 3787a2cc318bf98b2df6daaa01000000), Actor(RoLSubGrid, 3787a2cc318bf98b2df6daaa01000000))
(Actor(RoLSubGrid, 620c7863bd6d6f5d38c784c901000000), Actor(RoLSubGrid, 620c7863bd6d6f5d38c784c901000000), Actor(RoLSubGrid, 970ed42ec911b726a3f27e4e0

In [11]:
for it in range(64):
    roids = []
    for ix in range(dim):
        for iy in range(dim):
            roids.append(oids[ix,iy].step.remote())
    ray.get(roids)
    if (it%4==0):
        print("Step")
        print(ray.get(oids[0,0].get_grid.remote()))
        print(ray.get(oids[1,1].get_grid.remote()))
        
    
    roids = []
    for ix in range(dim):
        for iy in range(dim):
            if ix==iy:
                roids.append(oids[ix,iy].exchange.remote())
    ray.get(roids)
#    if (it%4==0):    
#        print(ray.get(oids[0,0].show.remote()))
#    print("Out of exchange")

Step
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


KeyboardInterrupt: 

(raylet) [2021-11-13 19:13:08,496 E 19090 1447252] agent_manager.cc:134: Not all required Ray dependencies for the runtime_env feature were found. To install the required dependencies, please run `pip install 'ray[default]'`.
(raylet) [2021-11-13 19:13:08,497 E 19090 1447252] worker_pool.cc:566: [Eagerly] Couldn't create a runtime environment for job 01000000.
(raylet) [2021-11-13 19:13:09,002 E 19090 1447252] agent_manager.cc:134: Not all required Ray dependencies for the runtime_env feature were found. To install the required dependencies, please run `pip install 'ray[default]'`.
(raylet) [2021-11-13 19:13:09,002 E 19090 1447252] worker_pool.cc:566: [Eagerly] Couldn't create a runtime environment for job 01000000.
(raylet) [2021-11-13 19:13:09,002 E 18535 1439576] agent_manager.cc:134: Not all required Ray dependencies for the runtime_env feature were found. To install the required dependencies, please run `pip install 'ray[default]'`.
(raylet) [2021-11-13 19:13:09,003 E 18535 143957

(pid=19097) 


In [6]:
ray.get(oids[0,0].exchange.remote())
print(ray.get(oids[0,0].get_grid.remote()))
ray.get(oids[1,1].exchange.remote())
print(ray.get(oids[1,1].get_grid.remote()))

# These hang... now why?

#ray.get(oids[0,1].exchange.remote())
#print(ray.get(oids[0,1].get_grid.remote()))

#ray.get(oids[1,0].exchange.remote())
#print(ray.get(oids[1,0].get_grid.remote()))

#ray.get(oids[1,1].exchange.remote())
#print(ray.get(oids[1,1].get_grid.remote()))


[[0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [ ]:
x = np.array(ray.get(oids[0,1].get_grid.remote()))

[top, bottom, left, right, tl, tr, bl, br] = ray.get(oids[0,1].get_neighbors.remote())


x[0,0] = ray.get(tl.cut.remote(7,8,7,8))  #tl cuts br
x[0,9] = ray.get(tr.cut.remote(7,8,0,1))  #tr cuts bl
x[9,0] = ray.get(bl.cut.remote(0,1,7,8))  #bl cuts tr
x[9,9] = ray.get(br.cut.remote(0,1,0,1))  #br cuts tl
x[0,1:9] = ray.get(top.cut.remote(7,8,0,8))  #top
x[9,1:9] = ray.get(bottom.cut.remote(0,1,0,8))  #bottom
x[1:9,0:1] = ray.get(left.cut.remote(0,8,7,8))  #left
x[1:9,9:10] = ray.get(right.cut.remote(0,8,0,1))  #right

print(x)

In [ ]:
x = np.array(ray.get(oids[1,1].get_grid.remote()))

x[0,0] = ray.get(oids[0,0].cut.remote(7,8,7,8))  #tl cuts br
x[0,9] = ray.get(oids[0,0].cut.remote(7,8,0,1))  #tr cuts bl
x[9,0] = ray.get(oids[0,0].cut.remote(0,1,7,8))  #bl cuts tr
x[9,9] = ray.get(oids[0,0].cut.remote(0,1,0,1))  #br cuts tl
x[0,1:9] = ray.get(oids[0,1].cut.remote(7,8,0,8))  #top
x[9,1:9] = ray.get(oids[0,1].cut.remote(0,1,0,8))  #bottom
x[1:9,0:1] = ray.get(oids[1,0].cut.remote(0,8,7,8))  #left
x[1:9,9:10] = ray.get(oids[1,0].cut.remote(0,8,0,1))  #right

print(x)

In [ ]:
funs = []
for ix in range(2):
    for iy in range(2):
        funs.append(oids[ix,iy].exchange.remote())
ray.get(funs)

for ix in range(dim):
    for iy in range(dim):
        print(ray.get(oids[ix,iy].get_grid.remote()))

In [ ]:
x = np.array(ray.get(oids[0,0].get_grid.remote()))

x[0,0] = ray.get(oids[1,1].cut.remote(7,8,7,8))  #tl cuts br
x[0,9] = ray.get(oids[0,1].cut.remote(7,8,0,1))  #tr cuts bl
x[9,0] = ray.get(oids[1,0].cut.remote(0,1,7,8))  #bl cuts tr
x[9,9] = ray.get(oids[1,1].cut.remote(0,1,0,1))  #br cuts tl
x[0,1:9] = ray.get(oids[1,0].cut.remote(7,8,0,8))  #top
x[9,1:9] = ray.get(oids[1,0].cut.remote(0,1,0,8))  #bottom
x[1:9,0:1] = ray.get(oids[0,1].cut.remote(0,8,7,8))  #left
x[1:9,9:10] = ray.get(oids[0,1].cut.remote(0,8,0,1))  #right

print(x)
y = ray.get(oids[0,0].exchange.remote())
print(ray.get(oids[0,0].get_grid.remote()))

In [ ]:
ray.get(oids[0,0].exchange.remote())
print(ray.get(oids[0,0].get_grid.remote()))

x = ray.get(oids[0,0].get_grid.remote())
x.shape
trimmed = x[1:-1,1:-1]
trimmed.shape
trimmed

In [ ]:
ray.get(oids[0,0].set_grid.remote(1))
ray.get(oids[0,1].set_grid.remote(2))  
ray.get(oids[1,0].set_grid.remote(3))  
ray.get(oids[1,1].set_grid.remote(4))  

ray.get(oids[0,0].exchange.remote())  
ray.get(oids[0,1].exchange.remote())  
ray.get(oids[1,0].exchange.remote())  
ray.get(oids[1,1].exchange.remote())  

print(ray.get(oids[0,0].get_grid.remote()))
print(ray.get(oids[0,1].get_grid.remote()))
print(ray.get(oids[1,0].get_grid.remote()))
print(ray.get(oids[1,1].get_grid.remote()))

In [ ]:
ray.shutdown()

In [ ]:
for it in range(16):
    funs = []
    for ix in range(dim):
        for iy in range(dim):
            funs.append(oids[ix,iy].step.remote())
    ray.get(funs)
    if (it%4==0):
        print(ray.get(oids[0,0].show.remote()))
    print("Out of step")
    
    funs = []
    for ix in range(dim):
        for iy in range(dim):
            funs.append(oids[ix,iy].exchange.remote())
    ray.get(funs)
    if (it%4==0):    
        print(ray.get(oids[0,0].show.remote()))
    print("Out of exchange")
    
#oids[0,0].step.remote()
#oids[0,0].exchange.remote()

#output = ray.get(oids[0,0].show.remote())

#print(ray.get(oids[0,0].show.remote()))

In [ ]:
for i in range(8):
    for i in range(4):
        #output=ray.get(oids[0,0].step.remote())
        oids[0,0].step.remote()
        print(ray.get(oids[0,0].show.remote()))

In [ ]:
import asyncio

@ray.remote(num_cpus=0)
class Barrier:
    def __init__(self, parties: int) -> None:
        self.parties = parties
        self._waiting: int
        self._event = asyncio.Event()

    def add_one(self) -> None:
        self._waiting += 1
        if self._waiting == self.parties:
            self._event.set()

    async def wait(self, worker: "Worker") -> None:
        """
        Wait until all we have at least `parties` waiters.
        """
        self.add_one()
        await self._event.wait()
        
barrier_oid = Barrier.remote(dim*dim)
print(barrier_oid)


In [ ]:
print("Step")
print(ray.get(oids[0,0].step.remote()))
print("Show")
print(ray.get(oids[0,0].show.remote()))
print("Step")
print(ray.get(oids[0,0].step.remote()))
print("Show")
print(ray.get(oids[0,0].show.remote()))


In [ ]:
soid = oids[0,0].show.remote()
print(ray.get(soid))

toid = oids[0,0].rules_of_life.remote()
print(ray.get(toid))

for i in range(8):
    print(ray.get(oids[0,0].step.remote()))


In [ ]:
dim = 8
local_grid = np.zeros(shape=[dim+2,dim+2],dtype=np.uint8)
glider = np.array([[0,1,0],[0,0,1],[1,1,1]], dtype=np.uint8)
local_grid[1:glider.shape[0]+1,1:glider.shape[1]+1] = glider
local_grid

In [ ]:
ray.shutdown()

In [ ]:
# this cell for debugging. remove ray.remote
dim = 2

oids = np.empty([dim,dim], dtype=object)

for iy in range(2):
    for ix in range(2):
        if ix == 0 and iy == 0:
            print(f"Here {ix} {iy}")
            oids[ix, iy] = RoLSubGrid(8,True)
            print((oids[ix,iy].show()))
        else:
            oids[ix, iy] = RoLSubGrid(8, False)
            print(f"Here {ix} {iy}")
            print((oids[ix,iy].show()))

for i in range(4):
    oids[0,0].step()
    print(oids[0,0].show())

In [ ]:
            
# T B L R TL BL TR BR            
for ix in range(dim):
    for iy in range(dim):
        print(f"top {(iy-1)%dim,ix}, bottom {(iy+1)%dim,ix}, left {iy,(ix-1)%dim}, right {iy,(ix+1)%dim}")
        print(f"tl {(iy-1)%dim,(ix-1)%dim}, bl {(iy+1)%dim,(ix-1)%dim}, tr {(iy-1)%dim,(ix+1)%dim}, br {(iy+1)%dim,(ix+1)%dim}")    
  
# set neighbors T B L R TL BL TR BR            
for ix in range(2):
    for iy in range(2):
        oids[ix,iy].set_neighbors( 
                    oids[(iy-1)%dim,ix], oids[(iy+1)%dim,ix], oids[iy,(ix-1)%dim], oids[iy,(ix+1)%dim],
                    oids[(iy-1)%dim,(ix-1)%dim], oids[(iy-1)%dim,(ix+1)%dim], oids[(iy+1)%dim,(ix-1)%dim],  oids[(iy+1)%dim,(ix+1)%dim]) 
        print(oids[ix,iy].get_neighbors())
                           

In [ ]:
## Ray debugging code

# T B L R TL TR BL BR            
#for ix in range(dim):
#    for iy in range(dim):
#        print(f"top {(iy-1)%dim,ix}, bottom {(iy+1)%dim,ix}, left {iy,(ix-1)%dim}, right {iy,(ix+1)%dim}")
#        print(f"tl {(iy-1)%dim,(ix-1)%dim}, tr {(iy-1)%dim,(ix+1)%dim}, bl {(iy+1)%dim,(ix-1)%dim},  br {(iy+1)%dim,(ix+1)%dim}")    

In [13]:
t = np.array([[0,0,0,0,0,0,0,0,0,0],
                   [ 0, 255, 1, 2, 3, 4, 5, 6, 7, 0],
                   [0, 10,11,12,13,14,15,16,17, 0],
                   [0, 20,21,22,23,24,25,26,27, 0],
                   [0, 30,31,32,33,34,35,36,37, 0],
                   [0, 0, 1, 2, 3, 4, 5, 6, 7, 0],
                   [0, 10,11,12,13,14,15,16,17,0],
                   [0, 20,21,22,23,24,25,26,27, 0],
                   [0, 30,31,32,33,34,35,36,37, 0],
                   [0,0,0,0,0,0,0,0,0,0]])